# FRTB PLAT: P&L Attribution Test

For FRTB IMA, the PLAT consist in comparing the Hypotetical P&L vs the Risk Theoretical P&L, with 2 tests:
1. Spearman correlation
2. KS (Kolmogorov–Smirnov) test

*Notes:* Details of calculations and thresholds: FRTB sections 32.34 - 32.42.

In [1]:
# import required libraries

import pandas as pd
import math as m
import numpy as np
import statistics as st

In [2]:
# import import PLAT.csv file into a pandas dataframe
plat_ts = pd.read_csv('PLAT.csv')
plat_ts

,Date,HPL,RTPL
0,23/11/2018,4.982745e+06,9.184258e+05
1,26/11/2018,3.035737e+06,4.055237e+06
2,27/11/2018,7.245941e+06,6.336971e+06
3,28/11/2018,8.727093e+06,6.210057e+06
4,29/11/2018,6.071703e+06,5.325492e+06
...,...,...,...
245,14/11/2019,9.245265e+06,3.061954e+06
246,15/11/2019,4.586715e+06,5.038695e+06
247,18/11/2019,9.861234e+05,1.039551e+06
248,19/11/2019,3.979911e+06,9.767369e+06


# 01. Spearman correlation

In [3]:
# 2.1 Spearman correlation
# 1) Rank
# 2) calculate HPL - RTPL covariance
# 3) RTPL & HPL std dev


# RANK HPL & RTPL

hpl = sorted(plat_ts['HPL'])
print('HPL sorted', hpl[:5])
print(type(hpl))

rtpl = sorted(plat_ts['RTPL'])
print('HPL sorted', rtpl[:5])
print(type(rtpl))

HPL sorted [2932.573947, 76774.45243, 80483.10113, 94238.9027, 198094.4426]
<class 'list'>
HPL sorted [7346.963745, 53241.15343, 90431.30126, 176278.9449, 256819.3709]
<class 'list'>


In [4]:
# Calculate ranked HPL-RTPL covariance
cov_matrix = np.cov(hpl, rtpl)
print(cov_matrix)

cov = cov_matrix[0][1] # CHECK: FIRST VALUE IS OK AS SpearCorr
print(cov)

[[8.27988459e+12 8.33207928e+12]
 [8.33207928e+12 8.42153492e+12]]
8332079283730.247


In [5]:
# HPL & RTPL standard deviation
hpl_stdev = st.stdev(hpl)
print(hpl_stdev)

rtpl_stdev = st.stdev(rtpl)
print(rtpl_stdev)

2877478.859851963
2901988.0982226394


In [6]:
# Spearman coef = Covariance(HPL, RTPL) / ( StDev(HPL) * StDev(RTPL) )

Sp_coef = cov / ( hpl_stdev * rtpl_stdev)
print('Spearman coef is', round(Sp_coef, 3) )


Spearman coef is 0.998


In [7]:
# stats library from scipy returns a similar value
from scipy import stats
Sp_coef_stats = stats.spearmanr(hpl, rtpl)[0]

print('Spearman coef with stats package is', Sp_coef_stats )

Spearman coef with stats package is 0.9999999999999998


# 02. KS test

In [8]:
# import normlibrary from scipy.stats to calculate empirical cumulative distribution function

from scipy.stats import norm

In [9]:
# calculate HPL & RTPL means

mean_hpl = st.mean(hpl)
print('HPL mean is', mean_hpl)

mean_rtpl = st.mean(rtpl)
print('RTPL mean is', mean_rtpl)

HPL mean is 4964451.925241228
RTPL mean is 5320435.69198734


In [10]:
# create new dataframde with ordered HPL & RTPL values
KS_df = pd.DataFrame( {'HPL': hpl, 'RTPL': rtpl })
KS_df

,HPL,RTPL
0,2.932574e+03,7.346964e+03
1,7.677445e+04,5.324115e+04
2,8.048310e+04,9.043130e+04
3,9.423890e+04,1.762789e+05
4,1.980944e+05,2.568194e+05
...,...,...
245,9.822712e+06,9.841759e+06
246,9.838982e+06,9.931843e+06
247,9.894501e+06,9.976193e+06
248,9.932360e+06,9.989574e+06


In [11]:
# calculate the empirical cumulative distribution function for every PnL value 

KS_df['HPL_ECDF'] = norm.cdf(hpl, mean_hpl, hpl_stdev)
KS_df['RTPL_ECDF'] = norm.cdf(rtpl, mean_rtpl, rtpl_stdev)
KS_df

,HPL,RTPL,HPL_ECDF,RTPL_ECDF
0,2.932574e+03,7.346964e+03,0.042331,0.033562
1,7.677445e+04,5.324115e+04,0.044698,0.034760
2,8.048310e+04,9.043130e+04,0.044819,0.035756
3,9.423890e+04,1.762789e+05,0.045273,0.038145
4,1.980944e+05,2.568194e+05,0.048817,0.040503
...,...,...,...,...
245,9.822712e+06,9.841759e+06,0.954330,0.940384
246,9.838982e+06,9.931843e+06,0.954870,0.943976
247,9.894501e+06,9.976193e+06,0.956673,0.945680
248,9.932360e+06,9.989574e+06,0.957869,0.946186


In [12]:
# calculate KS coef: maximum absolute difference of HPL & RTPL empirical cumulative distribution functions

KS_test = max( abs( KS_df['HPL_ECDF'] - KS_df['RTPL_ECDF'] ) )
print('Kolmogorov-Ssmirnov test metric is', KS_test)

Kolmogorov-Ssmirnov test metric is 0.05522747889421531


# PLAT color zone

Determine the PLAT color zone (FRTB - see sections 32.34 - 32.42 in the text)

In [13]:
# Determine PLAT color zone

In [14]:
# green
# - Spearman coef > 0.8, or
# - KS < 0.09

# red zone:
# - Spearman coef < 0.7, or
# - KS > 0.12

# amber
# - Spearman coef =< 0.8, or
# - KS > 0.09


In [15]:
if (Sp_coef > 0.8) & (KS_test < 0.09):
    PLAT_zone = 'green'
elif (Sp_coef < 0.7) or (KS_test > 0.12):
    PLAT_zone = 'red'
else:
    PLAT_zone = 'amber'

In [16]:
print('PLAT zone is:', PLAT_zone)

PLAT zone is: green
